In [1]:
import tensorflow as tf
import os
import numpy as np


c:\users\leonard\appdata\local\programs\python\python39\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
filepath = "\Dataset"
filename = "traindata.npy"
filename2 = "trainlabel.npy"

In [3]:
traindata = np.load(os.path.join(filepath,filename))
trainlabel = np.load(os.path.join(filepath, filename2))

In [15]:
trainlabel = tf.keras.utils.to_categorical(trainlabel)

[[1. 0.]
 [0. 1.]]


In [5]:
keras = tf.keras

In [6]:
batch_size = 64
epochs = 8
shuffle_buffer = 1000
train_data = tf.data.Dataset.from_tensor_slices((traindata, trainlabel))
train_data = train_data.shuffle(shuffle_buffer).batch(batch_size)

In [ ]:
base_model = keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(256,256,3))
base_model.summary()


In [8]:
base_model.trainable=False

In [11]:
print(train_data)

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>


In [9]:
poolinglayer = keras.layers.GlobalAveragePooling2D()
predictlayer = keras.layers.Dense(2,activation='softmax')

In [10]:
model = keras.Sequential([
    base_model,
    poolinglayer,
    predictlayer
])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [13]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.002),loss=keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [14]:
history = model.fit(train_data, epochs=epochs)
acc = history.history['accuracy']
print(acc)

Epoch 1/8
83/83 [==============================] - 421s 5s/step - loss: 0.5794 - accuracy: 0.7420
Epoch 2/8
83/83 [==============================] - 413s 5s/step - loss: 0.5010 - accuracy: 0.7643
Epoch 3/8
83/83 [==============================] - 398s 5s/step - loss: 0.4723 - accuracy: 0.7857
Epoch 4/8
83/83 [==============================] - 396s 5s/step - loss: 0.4446 - accuracy: 0.7963
Epoch 5/8
83/83 [==============================] - 406s 5s/step - loss: 0.4203 - accuracy: 0.8000
Epoch 6/8
83/83 [==============================] - 420s 5s/step - loss: 0.4064 - accuracy: 0.8103
Epoch 7/8
83/83 [==============================] - 428s 5s/step - loss: 0.3917 - accuracy: 0.8169
Epoch 8/8
83/83 [==============================] - 429s 5s/step - loss: 0.3847 - accuracy: 0.8243
[0.7419598698616028, 0.7642830014228821, 0.7856602072715759, 0.7962542772293091, 0.8000378608703613, 0.8102535009384155, 0.8168747425079346, 0.824252724647522]


In [28]:
test_data = np.load(os.path.join(filepath, "testdata.npy"))
test_label = np.load(os.path.join(filepath, "testlabel.npy"))
test_label = keras.utils.to_categorical(test_label)

In [25]:
model.evaluate(test_data, test_label)

20/20 [==============================] - 53s 3s/step - loss: 0.5249 - accuracy: 0.7115


[0.5249416828155518, 0.7115384340286255]

In [29]:
test_data = tf.data.Dataset.from_tensor_slices((test_data, test_label))
test_batches = test_data.batch(batch_size)

In [31]:
predict = model.predict(test_batches)

10/10 [==============================] - 46s 5s/step


In [35]:
print(predict[2])

[0.15614226 0.8438577 ]


In [42]:
class_names = ["Pneumonia", "Normal"]

In [46]:
model.save("Pneumonia_prediction.h5")